<a href="https://colab.research.google.com/github/hamzajg/movies-rec-colab/blob/master/MoviesRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-surprise

     |████████████████████████████████| 6.5MB 4.3MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678221 sha256=e8ba341cd9f212f997a0ff5709c9906a8d0c8f6181beee9223c0f93653cb717c
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [2]:
from MovieLens import MovieLens

ModuleNotFoundError: ignored